# **Project pRF fitting results to the Freesurfer surfaces**
This notebook will guide you through the step necessary to project fMRI pRF results on the surfaces that were generated with Freesurfer. It uses Freesurfer's `mri_vol2surf` method of projecting volumetric results to the surface. A nicer (newer) way of doing this, is with pycortex. There's a notebook in the `NHP-Freesurfer` repository called `freesurfer2pycortex` that explains that method. We reommend going that route.

### Initiation - Variables and paths

In [9]:
# what subject?
SUBJ=Danny

# where can we find the epi space the results are in?
# This is usually the reference from the `manual-masks' folder that is used for preprocessing and modelfit in NHP-BIDS.
EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-${SUBJ,}/func
EPI=${EPI_ROOT}/sub-${SUBJ,}_ref_func_res-1x1x1.nii.gz
EPI_MASK=${EPI_ROOT}/sub-${SUBJ,}_ref_func_mask_res-1x1x1.nii.gz
EPI_BRAIN=${EPI_ROOT}/sub-${SUBJ,}_ref_func_brain_res-1x1x1.nii.gz

# and the individual anatomical space they have been registered to
ANAT_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-${SUBJ,}/anat
ANAT=${EPI_ROOT}/sub-${SUBJ,}_ref_anat_res-1x1x1.nii.gz
ANAT_MASK=${ANAT_ROOT}/sub-${SUBJ,}_ref_anat_mask_res-1x1x1.nii.gz
ANAT_BRAIN=${ANAT_ROOT}/sub-${SUBJ,}_ref_anat_brain_res-1x1x1.nii.gz

# where is the anatomical you'd like to register to?
# $SUBJECTS_DIR should point to the Freesurfer subjects directory. Adjust your bashrc to export this if it doesn't do so already.
T1=${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz
# and the corresponding white matter segmentation?
WM=${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
# where's the surface folder for this subject?
FSSURF=${SUBJECTS_DIR}/${SUBJ}/surf
FSMMREG=${SUBJECTS_DIR}/${SUBJ}/manual-masks_toFS
mkdir -p $FSMMREG/anat
mkdir -p $FSMMREG/func

# where are the volume-based results
pRFVol=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/${SUBJ,}

# where will the surfaces be saved?
SURF_OUT=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/${SUBJ,}
mkdir -p ${SURF_OUT}

### Check if registrations of manual-mask references to FS-Subjects exist and create if they don't

In [6]:
# Check if a tkreg file has already been generated (if so, we can skip a few steps)
if [ -f "${FSMMREG}/func/reg.fsl.dat" ]
then
    make_tkreg=false
    echo "No need to create a tkreg file"
else
    make_tkreg=true
    echo "Create a tkreg file"
fi

Create a tkreg file


## Flirt registration of epi to T1
The T1 you pick here is the header adjusted one that formed the based of the surface generation in Freesurfer. We do not have to adjust the header of the epi, because flirt will generally be able to take of the scaling. If this fails for some reason you can still do it manually with `3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen <epi>.nii.gz`. Note that we set voxel sizes to 2 mm in the header in this case. The reason is that for the T1 we had 0.5 mm voxels and adjusted the header to 1 mm, a factor of 2. Our pre-processed epi's have 1 mm voxels, so applying the same factor of 2 means the header info should state that the voxels are 2 mm isotropic.

Be aware that, when you first adjust the header and then do flirt, the final registration matrix will not include this scaling and expect header-adjusted input. Any result you want to warp to the surface will thus first have to be adjusted. You don't have to do this is if the flirt is performed on the non-adjusted epi. 

NB1! One thing to carefully check is whether the x-direction of the voxel order matches between the epi and T1 because if it doesn't we will see left/right flips. With some of our older standard epi's this needs correction (later I corrected it in the reference file for pre-processing). There's a script that does that for you called `swap_xdir_voxels.sh`. You can find it in the `bin` folder of the `Process-NHP-MRI` repository (https://github.com/VisionandCognition/Process-NHP-MRI)

NB2! flirt works significantly better if you include the white matter segmentation (`-wmseg`) of the T1 and phase encoding direction (`-pedir`) of the epi. Check the documentation to find how to code this. If you used Chris Klink's standard epi sequence, your `-pedir` is `-2`.

## Create the tkregister matrix
Now we will use the flirt registration matrix to create a registration matrix in freesurfer format. To get there, we can use the freesurfer program `tkregister` that allows manual registration between 2 volumes. We will inititate it with the flirt matrix and check whether registration is good. If not, you can make manual adjustments, but flirt probably does a better job than manual attempts. Within `tkregister` you can check the alignment of the two volumes by clicking `compare`. Save the registration matrix as `reg.fsl.dat`.

In [7]:
# functional
if [ "$make_tkreg" = true ]; then # only do this if it doesn't exist already
    # get the brain and white matter volumes from freesurfer & convert to nifti
    mri_convert ${T1} ${SURF_OUT}/brain.nii.gz 
    mri_convert ${WM} ${SURF_OUT}/wm.nii.gz 
    # calculate the registration
    flirt -ref ${SURF_OUT}/brain.nii.gz -wmseg ${SURF_OUT}/wm.nii.gz -in ${EPI_BRAIN} \
        -out ${FSMMREG}/func/epi2anat.nii.gz -omat ${FSMMREG}/func/epi2anat.mat -pedir -2
    # create the tkreg matrix
    tkregister2 --mov ${EPI_BRAIN} --targ ${SURF_OUT}/brain.nii.gz --fsl  ${FSMMREG}/func/epi2anat.mat \
        --reg  ${FSMMREG}/func/reg.fsl.dat --s ${SUBJ}    
fi

mri_convert.bin /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/brainmask.mgz /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/danny/brain.nii.gz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/brainmask.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/danny/brain.nii.gz...
mri_convert.bin /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/wm.mgz /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/danny/wm.nii.gz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/wm.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/Doc

In [11]:
# anatomical
if [ "$make_tkreg" = true ]; then # only do this if it doesn't exist already
    # calculate the registration
    flirt -ref ${SURF_OUT}/brain.nii.gz -in ${ANAT_BRAIN} \
        -out ${FSMMREG}/anat/anat2anat.nii.gz -omat ${FSMMREG}/anat/anat2anat.mat
    # create the tkreg matrix
    tkregister2 --mov ${ANAT_BRAIN} --targ ${SURF_OUT}/brain.nii.gz --fsl  ${FSMMREG}/anat/anat2anat.mat \
        --reg  ${FSMMREG}/anat/reg.fsl.dat --s ${SUBJ}    
fi

tkregister_tcl /media/DATA1/NHP_MRI/freesurfer/tktools/tkregister2.tcl
---- FSL registration matrix --------
 1.99823   0.00424  -0.00501   40.11943;
-0.00098   0.00822  -2.00982   189.14003;
-0.00338   2.01534   0.01022   37.64399;
 0.00000   0.00000   0.00000   1.00000;
---------------------------------------
target  volume /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/danny/brain.nii.gz
movable volume /Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-danny/anat/sub-danny_ref_anat_brain_res-1x1x1.nii.gz
reg file       /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/manual-masks_toFS/anat/reg.fsl.dat
LoadVol        1
ZeroCRAS       0
$Id: tkregister2.c,v 1.132.2.1 2016/08/02 21:17:29 greve Exp $
Diagnostic Level -1
INFO: loading target /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/danny/brain.nii.gz
INFO: changing target type to float
Ttarg: --------------------
-1.00000   0.00000   0.

In [13]:
# copy the freesurfer surfaces to the outputfolder
cp -r ${FSSURF} ${SURF_OUT}/fs_surf
# copy the tkreg for volume to surface conversion as well
cp -r ${FSMMREG} ${SURF_OUT}/fs_tkreg

## Convert volumes to surface representation    
Cycle over models, cv-modes, hrfs, and R2-thresholds (for both hemispheres).

In [14]:
# values to cycle over
declare -a volspace=(anat)
declare -a hrfmod=(mhrf dhrf)
declare -a cv=(cv1)
declare -a models=(linhrf linhrf_neggain csshrf doghrf)
declare -a TH=(0 1 2 3 4 5 10)
# let the function automatically take care of both hemispheres internally

We can now convert the statistical volumes to surface representations using the `mri_vol2surf` command. Since this essentially brings a 3d result to 2d there is a choice to be made on how/where to sample. The `--projfrac` tells the command where between the WM/GM border (`--projfract 0`) and the pial surface (`--projfract 1`) to get the data. These fractions can also be negative (going into the WM) or higher than 1 (beyond the pial surface). Alternatively, you can average along the normal between WM/GM border and pial surface using `--projfrac-avg min max stepsize` or the maximum by using `--projfrac-max min max stepsize`.

In [19]:
# look at the mri_vol2surf documentation for more info
# mri_vol2surf --help

In [28]:
function Result_Vol2Surf() {
    SUBJ=$1
    VolSpace=$2
    HRF=$3
    CV=$4
    MOD=$5
    TH=$6
    SurfMethod=$7
    tkReg=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference/Surfaces/${SUBJ,}/fs_tkreg/${VolSpace}/reg.fsl.dat
    
    # get folder name
    if [ $MOD = linhrf_neggain ]; then
        modlabel=linhrf_${CV}_${HRF}_neggain
    else
        modlabel=${MOD}_${CV}_${HRF}
    fi
    
    # output folder depends on volspace
    if [ ${VolSpace} == "anat" ]; then 
        OUT=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/${SUBJ,}/${modlabel}/inAnat
    elif [ ${VolSpace} == "func" ]; then 
        OUT=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/${SUBJ,}/${modlabel}
    fi
    
    # R2 depends on CV mode (but in the end we only do cv1 anyway)
    if [ $CV = cv0 ]; then
        R2=${OUT}/Sess-${modlabel}_R2.nii.gz
    elif [ $CV = cv1 ]; then
        R2=${OUT}/Sess-${modlabel}_meanR2.nii.gz
    fi

    SURFOUT=${OUT}/TH_${TH}/surf
    mkdir -p ${SURFOUT}

    declare -a HEMI=(lh rh)

    # R2 to surface for both hemispheres
    for xh in ${HEMI[@]}; do
        # R2
        mri_vol2surf --trgsubject ${SUBJ} --src $R2 --out ${SURFOUT}/${xh}.R2.w \
            --out_type paint --${SurfMethod} \
            --srcreg ${tkReg} --hemi ${xh}
    done   

    # Threshold folder
    TH_fld=${OUT}/TH_${TH}
    TH_OUT=${TH_fld}/surf
    mkdir -p ${TH_OUT}
        
    # Volumes to variables == UNIVERSAL ONES ==
    ANG=${TH_fld}/ANG_th${TH}.nii.gz
    ECC=${TH_fld}/ECC_th${TH}.nii.gz
    FWHM=${TH_fld}/FWHM_th${TH}.nii.gz
    X=${TH_fld}/X_th${t}.nii.gz
    Y=${TH_fld}/Y_th${t}.nii.gz
    RFS=${TH_fld}/RFS_th${t}.nii.gz
    REAL=${TH_fld}/REAL_th${t}.nii.gz
    IMAG=${TH_fld}/IMAG_th${t}.nii.gz
    GAIN=${TH_fld}/GAIN_th${t}.nii.gz
                            
    # Change angles for visualisation
    fslmaths $ANG -sub 90 ${TH_fld}/ANGv.nii.gz
    fslmaths ${TH_fld}/ANGv.nii.gz -binv -mul 360 -add ${TH_fld}/ANGv.nii.gz ${TH_fld}/ANGv.nii.gz
    fslmaths ${TH_fld}/ANGv.nii.gz -mul ${TH_fld}/mask_th${t}.nii.gz -sub ${TH_fld}/invmask_th${t}.nii.gz ${TH_fld}/ANGv.nii.gz
    ANGV=${TH_fld}/ANGv.nii.gz

    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $ANG $ANG
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $ANGV $ANGV
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $ECC $ECC
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $FWHM $FWHM
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $X $X
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $Y $Y
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $RFS $RFS
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $REAL $REAL
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $IMAG $IMAG
    fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $GAIN $GAIN
               
    # create surface overlays                
    for xh in ${HEMI[@]}; do
        # angles
        mri_vol2surf --trgsubject ${SUBJ} --src $ANG --out ${SURFOUT}/${xh}.ANGLE.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        mri_vol2surf --trgsubject ${SUBJ} --src $ANGV --out ${SURFOUT}/${xh}.ANGLEV.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # ecc
        mri_vol2surf --trgsubject ${SUBJ} --src $ECC --out ${SURFOUT}/${xh}.ECC.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # fwhm
        mri_vol2surf --trgsubject ${SUBJ} --src $FWHM --out ${SURFOUT}/${xh}.FWHM.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # x
        mri_vol2surf --trgsubject ${SUBJ} --src $X --out ${SURFOUT}/${xh}.X.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # y
        mri_vol2surf --trgsubject ${SUBJ} --src $Y --out ${SURFOUT}/${xh}.Y.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # rfs
        mri_vol2surf --trgsubject ${SUBJ} --src $RFS --out ${SURFOUT}/${xh}.RFS.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # real
        mri_vol2surf --trgsubject ${SUBJ} --src $REAL --out ${SURFOUT}/${xh}.REAL.w \
            --out_type paint -${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # imag
        mri_vol2surf --trgsubject ${SUBJ} --src $IMAG --out ${SURFOUT}/${xh}.IMAG.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        # gain
        mri_vol2surf --trgsubject ${SUBJ} --src $GAIN --out ${SURFOUT}/${xh}.GAIN.w \
            --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
    done
    
    
    # CSS SPECIFIC
    if [ ${MOD:0:3} = 'css' ]; then
        EXPT=${TH_fld}/EXPT_th${t}.nii.gz
        EXPTSQ=${TH_fld}/EXPTSQ_th${t}.nii.gz
        fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $EXPT $EXPT
        fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $EXPT $EXPT
        for xh in ${HEMI[@]}; do
            mri_vol2surf --trgsubject ${SUBJ} --src $EXPT --out ${SURFOUT}/${xh}.EXPT.w \
                --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
            mri_vol2surf --trgsubject ${SUBJ} --src $EXPTSQ --out ${SURFOUT}/${xh}.EXPTSQ.w \
                --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        done
    fi        

    # DOG SPECIFIC
    if [ ${MOD:0:3} = 'dog' ]; then
        IRFS=${TH_fld}/IRFS_th${t}.nii.gz
        NAMP=${TH_fld}/NAMP_th${t}.nii.gz
        SDRATIO=${TH_fld}/SDRATIO_th${t}.nii.gz
        fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $IRFS $IRFS
        fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $NAMP $NAMP
        fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $SDRATIO $SDRATIO
        for xh in ${HEMI[@]}; do
            mri_vol2surf --trgsubject ${SUBJ} --src $IRFS --out ${SURFOUT}/${xh}.IRFS.w \
                --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
            mri_vol2surf --trgsubject ${SUBJ} --src $NAMP --out ${SURFOUT}/${xh}.NAMP.w \
                --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
            mri_vol2surf --trgsubject ${SUBJ} --src $SDRATIO --out ${SURFOUT}/${xh}.SDRATIO.w \
                --out_type paint --${SurfMethod} --srcreg ${tkReg} --hemi ${xh}
        done
    fi

}

## Choose the type of surface generation method     
We're going from volume (3d) to surface (2d), which means we're dropping a dimension. That can be done in several ways:
- take a point somewhere between the WM and pail borders (choose 'projfrac frac' with frac as the fraction from WM to pial)     
- take the maximum between the WM and pail borders (choose 'projfrac-max min max del')
- take the average between the WM and pail borders (choose 'projfrac-avg min max del')

In [29]:
SurfMethod="projfrac 0.7"
#SurfMethod="projfrac-avg 0 1 .1"
#SurfMethod="projfrac-max 0 1 .1"

In [30]:
# cycle over variable and execute Result_Vol2Surf
for v in ${volspace[@]}; do
    for h in ${hrfmod[@]}; do
        for c in ${cv[@]}; do
            for m in ${models[@]}; do
                for t in ${TH[@]}; do
                    Result_Vol2Surf ${SUBJ} $v $h $c $m $t $SurfMethod >/dev/null 2>&1
                done
            done
        done
    done
done

: 255

## Visualising the results  
The cells below provide the code for `tkSurfer`, but everything looks a lot more modern in `surfice`

In [ ]:
surfice

In [ ]:
# show in tksurfer (EXAMPLE)
xh=lh # switch hemisphere easily
tksurfer ${SUBJ} ${xh} graymid -patch full.patch.flat -overlay ${OUT}/${xh}.CT3_avg.w -overlay-reg ${tkReg}